In [1]:
import whisper

# This downloads the model and stores it locally
model = whisper.load_model("small")
print("Whisper Large Model downloaded successfully!")

Whisper Large Model downloaded successfully!


In [2]:
#!pip install -r requirements.txt
import soundfile as sf
import pyaudio
import wave
from openai import OpenAI
import whisper
import torch

!pip install ffmpeg-python

In [3]:
def record_audio(): #Function to record audio
       
    FORMAT = pyaudio.paInt16 # data format
    CHANNELS = 1 # mono, change to 2 if you want stereo
    RATE = 44100 # sample rate
    CHUNK = 1024 # frames per buffer
    RECORD_SECONDS = 25 # change this to the duration you want to record
    WAVE_OUTPUT_FILENAME = "output.wav"

    audio = pyaudio.PyAudio()

    # Start recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)
    print("Recording...")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Finished recording.")

    # Stop recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # Save the recorded data as a WAV file
    with wave.open(WAVE_OUTPUT_FILENAME, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))


In [4]:
#record_audio()

In [12]:
"""def convert_audio_to_text():
    # Load the audio file
    audio_file = "output.wav"
    audio, samplerate = sf.read(audio_file)

    # Convert the audio to text
    openai = OpenAI()
    text = openai.speech_to_text(audio, samplerate)

    print("Transcription:")
    print(text)"""

def convert_audio_to_text(audiofile):
    """Converts an audio file to text using the locally downloaded Whisper model."""
    
    # Load the locally stored Whisper large model
    model = whisper.load_model("medium")  # No API key required
    
    # Transcribe the audio
    result = model.transcribe(audiofile)
    #result = model.transcribe("your-audio-file.wav", )


    return result["text"]  # Extract and return transcribed text


In [6]:
import ffmpeg

def convert_mp3_to_wav(input_mp3, output_wav):
    """Converts an MP3 file to WAV format using FFmpeg."""
    try:
        ffmpeg.input(input_mp3).output(output_wav, ar=16000, ac=1, format='wav').run(overwrite_output=True)
        print(f"Conversion successful: {output_wav}")
    except Exception as e:
        print(f"Error during conversion: {e}")

In [10]:
def main():
   # record_audio()
    audio_path = "output.wav" 
    convert_mp3_to_wav(audio_path, "output.wav")
    transcribed_text = convert_audio_to_text("output.wav")
    print(transcribed_text)

In [14]:
main()

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1.1_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex

Error during conversion: ffmpeg error (see stderr output for detail)


/Users/akshat/anaconda3/envs/VirtualAI/lib/python3.11/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 My name is John Matthews and my date of birth is 17th February 2004. I am a salaried employee and my monthly income is 1.5 lakhs. The loan amount I am expecting is 20 lakhs and the purpose of this loan is a business loan.


In [16]:
import pyaudio
import wave
import threading

# Audio settings
FORMAT = pyaudio.paInt16  # 16-bit audio
CHANNELS = 1  # Mono audio
RATE = 44100  # Sample rate (Hz)
CHUNK = 1024  # Frames per buffer
WAVE_OUTPUT_FILENAME = "recorded_audio.wav"

# Initialize PyAudio
audio = pyaudio.PyAudio()
stream = None
frames = []
recording = False

def start_recording():
    """Start recording audio."""
    global stream, frames, recording
    if recording:
        print("Recording is already in progress...")
        return

    recording = True
    frames = []  # Clear previous recording

    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("🎙️ Recording started...")
    
    def record():
        while recording:
            data = stream.read(CHUNK)
            frames.append(data)

    # Start recording in a separate thread
    threading.Thread(target=record, daemon=True).start()

def stop_recording():
    """Stop recording and save the audio file."""
    global stream, frames, recording
    if not recording:
        print("No recording is in progress.")
        return

    recording = False  # Stop recording loop
    stream.stop_stream()
    stream.close()

    # Save the recording to a WAV file
    with wave.open(WAVE_OUTPUT_FILENAME, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print(f"✅ Recording stopped and saved as {WAVE_OUTPUT_FILENAME}")

# Example Usage
start_recording()  # Start recording
input("Press Enter to stop recording...\n")  # Wait for user input
stop_recording()  # Stop recording and save file

🎙️ Recording started...
✅ Recording stopped and saved as recorded_audio.wav
